In [1]:
from torch_geometric.loader import DataLoader
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/data/conghao001/anaconda3/envs/gnndrug/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.__version__

'1.10.2'

In [6]:
device = torch.device("cuda:2")
# device = torch.device("cpu")
device

device(type='cuda', index=2)

In [4]:
from utils import TestbedDataset
from models import GATNet_E, GATNet, GCNNet

In [7]:
model = GATNet()
model_path = 'root_folder/root_004/models/model_GAT-EP300-SW801010_GDSC.model'

# model = GCNNet()
# model_path = 'root_folder/root_003/models/model_GCN-EP300-SW801010_GDSC.model'
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [21]:
print(model)

GATNet(
  (gcn1): GATConv(192, 192, heads=10)
  (gcn2): GATConv(1920, 128, heads=1)
  (fc_g1): Linear(in_features=128, out_features=128, bias=True)
  (conv_xt_1): Conv1d(1, 32, kernel_size=(8,), stride=(1,))
  (pool_xt_1): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv_xt_2): Conv1d(32, 64, kernel_size=(8,), stride=(1,))
  (pool_xt_2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv_xt_3): Conv1d(64, 128, kernel_size=(8,), stride=(1,))
  (pool_xt_3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc1_xt): Linear(in_features=2944, out_features=128, bias=True)
  (fc1): Linear(in_features=256, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)


In [8]:
branch_folder = "root_folder/root_004"
dataset = 'GDSC'
test_data = TestbedDataset(root=branch_folder, dataset=dataset+'_test_mix')

Pre-processed data found: root_folder/root_004/processed/GDSC_test_mix.pt, loading ...


In [9]:
test_batch = 1
test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)

In [31]:
next(iter(test_loader))

DataBatch(x=[28, 192], edge_index=[2, 60], y=[1], edge_features=[60, 4], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 735], c_size=[1], batch=[28], ptr=[2])

In [22]:
i = 0
for data in test_loader:
    i += 1
    print(data)
    if i >= 5:
        break

DataBatch(x=[28, 192], edge_index=[2, 60], y=[1], edge_features=[60, 4], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 735], c_size=[1], batch=[28], ptr=[2])
DataBatch(x=[59, 192], edge_index=[2, 134], y=[1], edge_features=[134, 4], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 735], c_size=[1], batch=[59], ptr=[2])
DataBatch(x=[30, 192], edge_index=[2, 66], y=[1], edge_features=[66, 4], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 735], c_size=[1], batch=[30], ptr=[2])
DataBatch(x=[33, 192], edge_index=[2, 72], y=[1], edge_features=[72, 4], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 735], c_size=[1], batch=[33], ptr=[2])
DataBatch(x=[38, 192], edge_index=[2, 80], y=[1], edge_features=[80, 4], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 735], c_size=[1], batch=[38], ptr=[2])


In [10]:
from torch_geometric.nn import GNNExplainer

In [18]:
# test one drug
data = next(iter(test_loader))
print(data)

explainer = GNNExplainer(model, lr=1e-4, epochs=300)

x = data.x
edge_index = data.edge_index
y = data.y
batch_drug = data.batch
edge_features = data.edge_features
x_cell_mut = data.target

# model_args = (
#     x_cell_mut,
#     batch_drug,
#     edge_features
# )

kwargs = {
    "x_cell_mut": x_cell_mut,
    "batch_drug": batch_drug,
    "edge_features": edge_features
}

# node_feature_mask, edge_mask = explainer.explain_graph(x = x, edge_index = edge_index, x_cell_mut = x_cell_mut, batch_drug = batch_drug, edge_feat = edge_features)
node_feature_mask, edge_mask = explainer.explain_graph(x, edge_index, **kwargs)
ax, G = explainer.visualize_subgraph(edge_index, edge_mask, y=y)

DataBatch(x=[28, 192], edge_index=[2, 60], y=[1], edge_features=[60, 4], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 735], c_size=[1], batch=[28], ptr=[2])


TypeError: forward() got multiple values for argument 'x_cell_mut'

In [42]:
explainer = GNNExplainer(model, lr=1e-4, epochs=300)

for data in test_loader:
    x = data.x
    edge_index = data.edge_index
    y = data.y
    batch = data.batch
    edge_features = data.edge_features
    target = data.target
    
    node_feature_mask, edge_mask = explainer.explain_graph(x, edge_index, batch=batch, edge_feat = edge_features)
    ax, G = explainer.visualize_subgraph(edge_index, edge_mask, y=y)

TypeError: get_initial_prediction() got multiple values for keyword argument 'batch'